## KITTI 데이터셋에서 자동차 객체가 Occluded `0`인 이미지만 남기기
- Occluded `0`만 남기고 `1`, `2` 등의 라벨을 삭제한다 해도 라벨링 되지 않은 자동차가 배경에 있으면 오히려 학습에 안 좋은 영향을 줄 것으로 예상하여 온전히 가려지지 않은 자동차 객체만 있는 이미지를 학습했을 때 어떤 성능이 나오는지 확인하고자 진행

In [21]:
import os

os.chdir('/Users/Jinmi/Desktop')
original_dir = '/Users/Jinmi/Desktop/label_2'
original_dir_list = os.listdir(original_dir)
original_dir_list = sorted(original_dir_list)

### label -> ex_1
- DontCare 삭제
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '0'`만 있는 이미지만 남기기
- 'Car', 'Truck', 'Van' 세 개의 클래스에서 `Occluded '1'` 이상이 있다면 해당 이미지는 skip

In [22]:
# 새로 만들 라벨의 폴더
folder_name = 'label_ex_1'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

desktop = '/Users/Jinmi/Desktop'
os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_ex_1'

In [23]:
# 원본 라벨 확인
with open(os.path.join(original_dir, original_dir_list[2000]), 'r') as f:
    print(f.read())

Car 0.96 0 -0.86 0.00 201.43 303.98 369.00 1.50 1.78 3.69 -3.16 1.68 3.35 -1.56
Cyclist 0.00 1 -1.60 991.71 147.35 1029.66 217.34 1.72 0.78 1.71 10.47 0.90 18.34 -1.08
Car 0.00 1 -0.48 956.95 156.33 1124.69 205.06 1.51 1.75 4.45 14.38 0.76 23.55 0.06
Car 0.00 0 1.50 439.37 172.17 476.11 200.79 1.44 1.64 3.68 -7.94 1.02 37.94 1.29
Misc 0.00 0 1.76 370.74 169.93 394.17 203.93 1.87 0.86 2.12 -12.61 1.29 40.13 1.46
Car 0.00 1 1.92 224.79 176.21 310.48 218.48 1.63 1.78 4.54 -14.14 1.46 29.92 1.49
DontCare -1 -1 -10 874.10 153.15 911.13 194.29 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 621.70 163.74 632.35 175.85 -1 -1 -1 -1000 -1000 -1000 -10



In [24]:
original_dir_list[:10]

['000000.txt',
 '000001.txt',
 '000002.txt',
 '000003.txt',
 '000004.txt',
 '000005.txt',
 '000006.txt',
 '000007.txt',
 '000008.txt',
 '000009.txt']

In [25]:
len(original_dir_list)

7481

In [26]:
original_dir_list[-1]

'007480.txt'

#### 라벨 가려내기

In [27]:
%%time
to_path = os.path.join(desktop, folder_name)
for txt in original_dir_list:
    with open(os.path.join(original_dir, txt), 'r') as fr:
        lines = []
        count = 0 # 기준으로 삼을 count. count 1 이상이면 해당 라벨은 skip할 예정
        for line in fr:
            word_list = line.split()
            
            # DontCare 제외
            if word_list[0] == 'DontCare':
                continue
                
            # 세 개의 클래스 중 하나면서 Occluded 0 아니면 count 올리기    
            if (word_list[2] != '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                count += 1
                continue
            
            # 나머지면 추가
            lines.append(line)
        if count > 0:
            continue
            
        # 자동차 객체에서 Occlusion 0만 있는 라벨은 새롭게 저장하기   
        else:
            with open(os.path.join(to_path, txt), 'w') as fw:
                fw.writelines(lines)       

CPU times: user 832 ms, sys: 1.34 s, total: 2.17 s
Wall time: 3.95 s


In [28]:
# 개수 확인
len(os.listdir(to_path))

2388

In [30]:
# 추려낸 것들 리스트로 모으기
extracted_list = os.listdir(to_path)

In [38]:
# 변경된 라벨 확인
for i in range(20, 40):
    with open(os.path.join(original_dir, extracted_list[i]), 'r') as f2:
        print('\toriginal')
        print(f2.read())
    with open(os.path.join(to_path, extracted_list[i]), 'r') as f:
        print(f'\tmodified, {extracted_list[i]}')        
        print(f.read())
        print('\n====\n')

	original
Car 0.00 0 1.57 607.86 189.59 626.06 206.47 1.41 1.59 4.47 0.61 2.98 65.83 1.58
DontCare -1 -1 -10 648.10 190.65 664.81 202.15 -1 -1 -1 -1000 -1000 -1000 -10

	modified, 005896.txt
Car 0.00 0 1.57 607.86 189.59 626.06 206.47 1.41 1.59 4.47 0.61 2.98 65.83 1.58


====

	original
Car 0.00 0 1.43 804.94 177.15 1056.88 372.98 1.57 1.73 4.15 3.13 1.63 7.99 1.79
DontCare -1 -1 -10 664.77 202.10 680.44 214.64 -1 -1 -1 -1000 -1000 -1000 -10

	modified, 006347.txt
Car 0.00 0 1.43 804.94 177.15 1056.88 372.98 1.57 1.73 4.15 3.13 1.63 7.99 1.79


====

	original
Cyclist 0.00 3 -1.75 187.33 165.34 210.73 202.65 1.86 0.71 1.76 -20.80 1.11 36.21 -2.27
Pedestrian 0.00 0 -1.90 441.09 161.10 486.42 279.52 1.72 0.57 0.99 -2.20 1.44 10.83 -2.09
Pedestrian 0.00 0 -1.91 389.84 147.59 436.17 282.60 1.90 0.61 1.11 -2.79 1.43 10.57 -2.16
Pedestrian 0.00 0 1.06 536.47 154.40 582.14 295.43 1.79 0.79 0.96 -0.68 1.46 9.60 1.00
Cyclist 0.92 3 1.84 1216.26 160.58 1223.00 258.28 1.70 0.60 1.70 11.54 1.34 1

In [39]:
# 위에서 포함되지 않은 라벨들 모아서 작업이 제대로 진행됐는지 확인하기
not_included = []

for txt in original_dir_list:
    with open(os.path.join(original_dir, txt), 'r') as fr:
        count = 0
        for line in fr:
            word_list = line.split()
            if (word_list[2] != '0') and ((word_list[0] == 'Car') or (word_list[0] == 'Van') or (word_list[0] == 'Truck')):
                count += 1
        if count > 0:
            not_included.append(txt)

In [41]:
# 개수 확인
print(len(not_included))
print(len(extracted_list))
print(len(extracted_list) + len(not_included))

5093
2388
7481


In [43]:
answer = True
for txt in extracted_list:
    if txt in not_included:
        answer = False
        
print(answer)

True


### YOLOv5 포맷으로 변경

In [44]:
try_path = '/Users/Jinmi/Desktop/yolo_labels_ex_1'
if not os.path.exists(try_path):
    os.mkdir(try_path)

In [45]:
import cv2
import pandas as pd

# 클래스 값을 숫자로 변환해주는 매핑 함수
def label_mapping(string):
    map_dict = {'Car' : 0,
                'Pedestrian' : 1,
                'Truck' : 2,
                'Cyclist' : 3,
                'Van' : 4,
                'Tram' : 5,
                'Misc' : 6,
                'Person_sitting' : 7,
                'DontCare' : 8,
                }
    return map_dict[string]

raw_label_root_path = '/Users/Jinmi/Desktop/label_ex_1'
raw_img_root_path = '/Users/Jinmi/Desktop/images/training/image_2'
label_filenames = os.listdir(raw_label_root_path)
to_path = try_path

# 라벨 새로 저장할 경로 생성
if not os.path.exists(to_path):
    os.mkdir(to_path)

# 변환 시 직관적으로 알 수 있게끔 해줄 판다스 데이터프레임 헤더 리스트
columns = ['type', 'truncated', 'occluded', 'alpha', 'bbox_l', 'bbox_t', 'bbox_r', 'bbox_b', '3d_h', '3d_w', '3d_l', '3d_x', '3d_y', '3d_z', '3d_rot_y', 'score']

# 이미지 경로 전체 돌면서 매칭되는 기존 라벨값 불러온 후 변환 및 다른 경로에 저장
for filename in label_filenames:
    lab_path = os.path.join(raw_label_root_path, filename)
    img_path = os.path.join(raw_img_root_path, filename[:6]+'.png')
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height = img.shape[0]
    width = img.shape[1]
    img_id = img_path[-10:-4] # 이미지 넘버
    df = pd.read_csv(lab_path, sep=' ', names=columns)

    df['type'] = df['type'].map(label_mapping)
    df = df[df['type'] != 8] # 혹시 모를 DontCare 제외
    # df = df[(df['occluded'] == 1) | (df['occluded'] == 2)]
    df['ob_width'] = df['bbox_r'] - df['bbox_l']
    df['ob_height'] = df['bbox_b'] - df['bbox_t']
    df['x_center'] = df['ob_width'] / 2 + df['bbox_l']
    df['y_center'] = df['ob_height'] / 2 + df['bbox_t']
    df['norm_x_center'] = df['x_center'] / width
    df['norm_y_center'] = df['y_center'] / height
    df['norm_ob_width'] = df['ob_width'] / width
    df['norm_ob_height'] = df['ob_height'] / height

    df_to_save = df.loc[:, ['type', 'norm_x_center', 'norm_y_center', 'norm_ob_width', 'norm_ob_height']]
    df_to_save.to_csv(os.path.join(to_path, img_id + '.txt'), sep=' ', index=False, header=False)

In [46]:
len(os.listdir(try_path))

2388

In [48]:
extracted_list[-10:]

['004005.txt',
 '006606.txt',
 '001169.txt',
 '003018.txt',
 '001627.txt',
 '005441.txt',
 '004987.txt',
 '004039.txt',
 '001155.txt',
 '003024.txt']

### 나머지 약 5000장도 DontCare 없애고 YOLO 버전으로 바꾸기

In [57]:
os.chdir('..')

In [58]:
folder_name = 'label_ex_1_etc'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

desktop = '/Users/Jinmi/Desktop'
os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/label_ex_1_etc'

In [59]:
%%time
to_path = os.path.join(desktop, folder_name)
for txt in not_included:
    with open(os.path.join(original_dir, txt), 'r') as fr:
        lines = []
        for line in fr:
            word_list = line.split()
            
            # DontCare 제외
            if word_list[0] == 'DontCare':
                continue
            
            # 나머지면 추가
            lines.append(line)
        with open(os.path.join(to_path, txt), 'w') as fw:
            fw.writelines(lines)       

CPU times: user 722 ms, sys: 1.58 s, total: 2.3 s
Wall time: 4.16 s


In [61]:
# 변경된 라벨 확인
for i in range(10):
    with open(os.path.join(original_dir, not_included[i]), 'r') as f2:
        print('\toriginal')
        print(f2.read())
    with open(os.path.join(to_path, not_included[i]), 'r') as f:
        print(f'\tmodified, {not_included[i]}')        
        print(f.read())
        print('\n====\n')

	original
Car 0.00 2 -1.55 548.00 171.33 572.40 194.42 1.48 1.56 3.62 -2.72 0.82 48.22 -1.62
Car 0.00 0 -1.21 505.25 168.37 575.44 209.18 1.67 1.64 4.32 -2.61 1.13 31.73 -1.30
Car 0.00 0 0.15 49.70 185.65 227.42 246.96 1.50 1.62 3.88 -12.54 1.64 19.72 -0.42
Car 0.00 1 2.05 328.67 170.65 397.24 204.16 1.68 1.67 4.29 -12.66 1.13 38.44 1.73
DontCare -1 -1 -10 603.36 169.62 631.06 186.56 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 578.97 168.88 603.78 187.56 -1 -1 -1 -1000 -1000 -1000 -10

	modified, 000006.txt
Car 0.00 2 -1.55 548.00 171.33 572.40 194.42 1.48 1.56 3.62 -2.72 0.82 48.22 -1.62
Car 0.00 0 -1.21 505.25 168.37 575.44 209.18 1.67 1.64 4.32 -2.61 1.13 31.73 -1.30
Car 0.00 0 0.15 49.70 185.65 227.42 246.96 1.50 1.62 3.88 -12.54 1.64 19.72 -0.42
Car 0.00 1 2.05 328.67 170.65 397.24 204.16 1.68 1.67 4.29 -12.66 1.13 38.44 1.73


====

	original
Car 0.88 3 -0.69 0.00 192.37 402.31 374.00 1.60 1.57 3.23 -2.70 1.74 3.68 -1.29
Car 0.00 1 2.04 334.85 178.94 624.50 372.04 1.57 1.50

In [74]:
try_path = '/Users/Jinmi/Desktop/yolo_labels_ex_1_ect'
if not os.path.exists(try_path):
    os.mkdir(try_path)

In [75]:
import cv2
import pandas as pd

# 클래스 값을 숫자로 변환해주는 매핑 함수
def label_mapping(string):
    map_dict = {'Car' : 0,
                'Pedestrian' : 1,
                'Truck' : 2,
                'Cyclist' : 3,
                'Van' : 4,
                'Tram' : 5,
                'Misc' : 6,
                'Person_sitting' : 7,
                'DontCare' : 8,
                }
    return map_dict[string]

raw_label_root_path = '/Users/Jinmi/Desktop/label_ex_1_etc'
raw_img_root_path = '/Users/Jinmi/Desktop/images/training/image_2'
label_filenames = os.listdir(raw_label_root_path)
to_path = try_path

# 라벨 새로 저장할 경로 생성
if not os.path.exists(to_path):
    os.mkdir(to_path)

# 변환 시 직관적으로 알 수 있게끔 해줄 판다스 데이터프레임 헤더 리스트
columns = ['type', 'truncated', 'occluded', 'alpha', 'bbox_l', 'bbox_t', 'bbox_r', 'bbox_b', '3d_h', '3d_w', '3d_l', '3d_x', '3d_y', '3d_z', '3d_rot_y', 'score']

# 이미지 경로 전체 돌면서 매칭되는 기존 라벨값 불러온 후 변환 및 다른 경로에 저장
for filename in label_filenames:
    lab_path = os.path.join(raw_label_root_path, filename)
    img_path = os.path.join(raw_img_root_path, filename[:6]+'.png')
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    height = img.shape[0]
    width = img.shape[1]
    img_id = img_path[-10:-4] # 이미지 넘버
    df = pd.read_csv(lab_path, sep=' ', names=columns)

    df['type'] = df['type'].map(label_mapping)
    df = df[df['type'] != 8] # 혹시 모를 DontCare 제외
    # df = df[(df['occluded'] == 1) | (df['occluded'] == 2)]
    df['ob_width'] = df['bbox_r'] - df['bbox_l']
    df['ob_height'] = df['bbox_b'] - df['bbox_t']
    df['x_center'] = df['ob_width'] / 2 + df['bbox_l']
    df['y_center'] = df['ob_height'] / 2 + df['bbox_t']
    df['norm_x_center'] = df['x_center'] / width
    df['norm_y_center'] = df['y_center'] / height
    df['norm_ob_width'] = df['ob_width'] / width
    df['norm_ob_height'] = df['ob_height'] / height

    df_to_save = df.loc[:, ['type', 'norm_x_center', 'norm_y_center', 'norm_ob_width', 'norm_ob_height']]
    df_to_save.to_csv(os.path.join(to_path, img_id + '.txt'), sep=' ', index=False, header=False)

### val, test 라벨 떼어놓기

In [88]:
desktop = '/Users/Jinmi/Desktop'
os.chdir(desktop)

folder_name = 'yolo_label_ex_1_val'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

desktop = '/Users/Jinmi/Desktop'
os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/yolo_label_ex_1_val'

In [91]:
import shutil

for txt in not_included[:300]:
    shutil.copy(os.path.join('/Users/Jinmi/Desktop/yolo_labels_ex_1_etc', txt),
               os.path.join(desktop, folder_name, txt))

In [92]:
len(os.listdir(os.path.join(desktop, folder_name)))

300

In [93]:
os.chdir('..')
folder_name = 'yolo_label_ex_1_test'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

desktop = '/Users/Jinmi/Desktop'
os.chdir(os.path.join(desktop, folder_name))
os.getcwd()

'/Users/Jinmi/Desktop/yolo_label_ex_1_test'

In [95]:
import shutil

for txt in not_included[300:600]:
    shutil.copy(os.path.join('/Users/Jinmi/Desktop/yolo_labels_ex_1_etc', txt),
               os.path.join(desktop, folder_name, txt))

In [96]:
len(os.listdir(os.path.join(desktop, folder_name)))

300